In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import h5py
subhalo = h5py.File('/store/clues/HESTIA/RE_SIMS/8192/GAL_FOR/17_11/output_2x2.5Mpc/groups_127/fof_subhalo_tab_127.0.hdf5','r')

def printname(name):
    print (name)
    
subhalo.visit(printname)

Config
Group
Group/GroupBHMass
Group/GroupBHMdot
Group/GroupCM
Group/GroupFirstSub
Group/GroupGasMetalFractions
Group/GroupGasMetallicity
Group/GroupLen
Group/GroupLenType
Group/GroupMass
Group/GroupMassType
Group/GroupNsubs
Group/GroupPos
Group/GroupRadioLuminosity
Group/GroupSFR
Group/GroupStarMetalFractions
Group/GroupStarMetallicity
Group/GroupVel
Group/GroupWindMass
Group/GroupXrayLuminosity
Group/Group_M_Crit200
Group/Group_M_Crit500
Group/Group_M_Mean200
Group/Group_M_TopHat200
Group/Group_R_Crit200
Group/Group_R_Crit500
Group/Group_R_Mean200
Group/Group_R_TopHat200
Header
IDs
Parameters
Subhalo
Subhalo/SubhaloBHMass
Subhalo/SubhaloBHMdot
Subhalo/SubhaloBfldDisk
Subhalo/SubhaloBfldHalo
Subhalo/SubhaloCM
Subhalo/SubhaloGasMetalFractions
Subhalo/SubhaloGasMetalFractionsHalfRad
Subhalo/SubhaloGasMetalFractionsMaxRad
Subhalo/SubhaloGasMetalFractionsSfr
Subhalo/SubhaloGasMetalFractionsSfrWeighted
Subhalo/SubhaloGasMetallicity
Subhalo/SubhaloGasMetallicityHalfRad
Subhalo/SubhaloGasMet

In [ ]:
%%time
import gc
import joblib
import h5py
import pathlib
import numpy as np

dir_path = pathlib.Path('/store/clues/HESTIA/RE_SIMS/8192/GAL_FOR/17_11/output_2x2.5Mpc/snapdir_127')


def load(path):
    with h5py.File(path,'r') as snap:
        pstr = snap['PartType4/Coordinates'][()]
        pdrk = snap['PartType1/Coordinates'][()]
        pgas = snap['PartType0/Coordinates'][()]
#         mstr = snap['PartType4/Masses'][()]
        vstr = snap['PartType4/Velocities'][()]

    return {'StrPos': pstr, 'DrkPos': pdrk, 'GasPos': pgas, 'StrVel': vstr}

with joblib.Parallel(n_jobs=8) as P:
    results = P(
        joblib.delayed(load)(path)
        for path in dir_path.glob('*.hdf5'))
    
# results = np.vstack(results)
gc.collect()


pos_str = np.concatenate([r['StrPos']for r in results])
pos_drk = np.concatenate([r['DrkPos']for r in results])
pos_gas = np.concatenate([r['GasPos']for r in results])
vel_str = np.concatenate([r['StrVel']for r in results])

In [ ]:



G = 4.299e-6 #kpc*(km/s)*Msol^-1

r = np.concatenate(rstr,rgas,rdrk)
m = np.concatenate(mstr,mgas,mdrk)

psort = np.argsort(r)

vol  = 4./3 * np.pi * r[psort]**3
mass = m[psort]
rho  = mass / vol

rho_crit = 3*H**2 / (8*np.pi*G)

mask, = np.where(rho < 200*rho_crit)
Mvir = mvir[-1]

In [7]:
3.2404e-17*3.2e7*1e9

1.036928

In [ ]:
h = 0.6777
G = 4.299e-6 #kpc*(km/s)*Msol^-1
a0 = 1
H0 = h*100
omega_lambda = 0.682
omega_matter = 0.318
omega0 = omega_matter + omega_lambda

z = a0/aexp[isnap] - 1.
Ht = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)
time = tiempo.conv(z, h, omega_lambda, omega_matter)

#---aca paso las coordenadas respecto al centro de la galaxia------
xstr = (pstr[:,0]-cm[0])*aexp[isnap]/h
ystr = (pstr[:,1]-cm[1])*aexp[isnap]/h
zstr = (pstr[:,2]-cm[2])*aexp[isnap]/h

v_x = vel[:,0] *np.sqrt(aexp[isnap]) + Ht *xstr/1000.
v_y = vel[:,1] *np.sqrt(aexp[isnap]) + Ht *ystr/1000.
v_z = vel[:,2] *np.sqrt(aexp[isnap]) + Ht *zstr/1000.